In [1]:
import os
import sys 
sys.path.insert(0, os.path.abspath('..\..\..\..\sysmpy'))  
from sysmpy import *

# 1. Define a simple system requirement 

In [2]:
r1 = Requirement("Req 1_1", des="The system shall fit into a size not exceeding 4", range=[1, 2, 3, 4])

# 2. Define an action model

                +---------------+
                |               |
                |     start     |
                |               |
                +---------------+
                        | process 1
                        |
                +---------------+
                |               |
                |   Action1_1   |
                |               |        
                +---------------+        
                        |
                        |
                +---------------+
                |               |
                |   Action1_2   |
                |               |        
                +---------------+        
                        |
                        | 
                +---------------+
                |               |
                |      End      |
                |               |
                +---------------+


In [3]:
p = Process("process")
act1 = p.Action("Action1")
act2 = p.Action("Action2")
act3 = p.Action("Action3")

# 3. Define a physical model (i.e., Block Diagram)

In [4]:
# Define components
c1 = Component("Com1", des="This is a system")
c1_1 = c1.Component("Com1_1", des="This is another component")
c1_2 = c1.Component("Com1_2", des="This is another component")

# Make relationships between components and processes/actions
c1.performs(p)
c1_1.performs(act1)
c1_2.performs(act2)

# Make properties for components
pro1 = c1.Property("Total size")
pro1_1 = c1_1.Property("Size", range=[1, 2, 3])
pro1_2 = c1_2.Property("Size", range=[1, 2, 3])

# 4. Define a link between components and requirements

                +---------------+
                |               |
                |     Com1      |
                |               |
                +---------------+
                        O
                        |  
                        |                      
                +---------------+        
                |               |       
                |    Req 1_1    |       
                |               |        
                +---------------+   
                

In [5]:
# Make relationships between components and requirements
pro1.traced_from(r1)

# 5. Perform ExSim (i.e., model center)
                    +---------------+   
                    |               |
                    |    Com 1_1    |
                    |               |
                    +---------------+
                    |     Size      |
                    +---------------+          
                                   \
                                    \
        +---------------+            +--------------------+            +---------------+
        |               |            |       Action       |            |               |
        |     Start     |----------->|        for         |----------->|      End      |
        |               |            |  size calculation  |            |               |
        +---------------+            +--------------------+            +---------------+
                                    /                      \   
                                   /                        \
                    +---------------+                       +---------------+
                    |               |                       |               |
                    |    Com 1_2    |                       |     Com 1     |
                    |               |                       |               |
                    +---------------+                       +---------------+
                    |     Size      |                       |   Total size  |
                    +---------------+                       +---------------+

In [6]:
# Define processes and actions
p_ex = Process("ExSim process 1")
act_ex = p_ex.Action('size calculator')

act_ex.receives(pro1_1)
act_ex.receives(pro1_2)
act_ex.sends(pro1)


# External simulations Script  
def exsim_function1(io):
    i1, i2, out = io.get("Com1_1.Size"), io.get("Com1_2.Size"), io.get("Com1.Total size")
    i1_value = i1.get_random_value()
    i2_value = i2.get_random_value()
    
    print(f"{i1.value}, {i2.value}")

    out.value = i1_value + i2_value

    print("out: " + str(out.value))
    
# Script end  
act_ex.func(exsim_function1)

await p_ex.sim(print_out=True)

The simulation is connected to the web distributor.
At Time 0, "size calculator" waiting.
At Time 0, "size calculator" activated.
3, 2
out: 5
At Time 1, "size calculator" completed.
At Time 1, "size calculator" waiting.
At Time 1, "ExSim process 1_END" simulation 0.
At Time 1, "size calculator" activated.
1, 1
out: 2
At Time 2, "size calculator" completed.
At Time 2, "size calculator" waiting.
At Time 2, "ExSim process 1_END" simulation 1.
At Time 2, "size calculator" activated.
1, 3
out: 4
At Time 3, "size calculator" completed.
At Time 3, "size calculator" waiting.
At Time 3, "ExSim process 1_END" simulation 2.
At Time 3, "size calculator" activated.
1, 1
out: 2
At Time 4, "size calculator" completed.
At Time 4, "size calculator" waiting.
At Time 4, "ExSim process 1_END" simulation 3.
At Time 4, "size calculator" activated.
2, 1
out: 3
At Time 5, "size calculator" completed.
At Time 5, "size calculator" waiting.
At Time 5, "ExSim process 1_END" simulation 4.
--------- Simulation Comp

# 6. Check requirements with the simulation results

In [7]:
p_ex.evaluate_requirements()

Req 1_1 Range is [1, 2, 3, 4]: {'Total size': 3} passed
